In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/train.csv.gz")
# SalePrice 컬럼에 로그 변환 적용
df['SalePrice'] = np.log1p(df['SalePrice'])

print("SalePrice 로그 변환 완료")
print("SalePrice 기술 통계량:")
print(df['SalePrice'].describe())



SalePrice 로그 변환 완료
SalePrice 기술 통계량:
count    1460.000000
mean       12.024057
std         0.399449
min        10.460271
25%        11.775105
50%        12.001512
75%        12.273736
max        13.534474
Name: SalePrice, dtype: float64


In [10]:
import numpy as np

# SalePrice와의 피어슨 상관계수 계산
corr_matrix = df.corr(numeric_only=True)
saleprice_corr = corr_matrix['SalePrice'].sort_values(ascending=False)

print("각 컬럼과 SalePrice의 피어슨 상관계수:")
print(saleprice_corr)

# 더 많은 피처를 포함시키기 위해 상관계수 0.5 이상인 피처 선택
threshold = 0.5
selected_features = saleprice_corr[abs(saleprice_corr) >= threshold].index.tolist()
selected_features.remove('SalePrice')  # 타겟 컬럼은 제외

print("\n모델링에 사용할 feature (상관계수 |0.5| 이상):")
print(selected_features)


각 컬럼과 SalePrice의 피어슨 상관계수:
SalePrice        1.000000
OverallQual      0.817185
GrLivArea        0.700927
GarageCars       0.680625
GarageArea       0.650888
TotalBsmtSF      0.612134
1stFlrSF         0.596981
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
GarageYrBlt      0.541073
TotRmsAbvGrd     0.534422
Fireplaces       0.489450
MasVnrArea       0.430809
BsmtFinSF1       0.372023
LotFrontage      0.355879
WoodDeckSF       0.334135
OpenPorchSF      0.321053
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209043
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057330
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF    -0.037963
MSSubClass      -0.073959
KitchenAbvGr    -0.147548
EnclosedPor

In [11]:
# selected_features의 요소 갯수 출력
print("selected_features의 요소 갯수:", len(selected_features))


selected_features의 요소 갯수: 11


In [12]:
# selected_features의 타입 확인
print("selected_features 타입:", type(selected_features))
print("selected_features:", selected_features)

# 선택된 feature와 타겟 변수만 추출
target = 'SalePrice'
features = selected_features.copy()
columns_to_include = features + [target] 
df_selected = df[columns_to_include].copy()

# 결측치 확인
print("\n각 feature별 결측치 개수:")
print(df_selected.isnull().sum())


selected_features 타입: <class 'list'>
selected_features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'TotRmsAbvGrd']

각 feature별 결측치 개수:
OverallQual      0
GrLivArea        0
GarageCars       0
GarageArea       0
TotalBsmtSF      0
1stFlrSF         0
FullBath         0
YearBuilt        0
YearRemodAdd     0
GarageYrBlt     81
TotRmsAbvGrd     0
SalePrice        0
dtype: int64


In [13]:

# 결측치 처리 (수치형: 평균, 범주형: 최빈값)
for col in selected_features:
    if df_selected[col].dtype == 'object':
        # 범주형: 최빈값으로 대체
        mode = df_selected[col].mode()[0]
        df_selected[col] = df_selected[col].fillna(mode)
    else:
        # 수치형: 평균으로 대체
        mean = df_selected[col].mean()
        df_selected[col] = df_selected[col].fillna(mean)


In [14]:

# 인코딩 (범주형 변수만)
from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in selected_features:
    if df_selected[col].dtype == 'object':
        le = LabelEncoder()
        df_selected[col] = le.fit_transform(df_selected[col])

# 정규화/스케일링 (수치형 변수만)
scaler = StandardScaler()
num_cols = [col for col in selected_features if df_selected[col].dtype != 'object']
df_selected[num_cols] = scaler.fit_transform(df_selected[num_cols])

# 최종 데이터프레임 확인
print("\n전처리된 데이터프레임 info:")
print(df_selected.info())




전처리된 데이터프레임 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1460 non-null   float64
 1   GrLivArea     1460 non-null   float64
 2   GarageCars    1460 non-null   float64
 3   GarageArea    1460 non-null   float64
 4   TotalBsmtSF   1460 non-null   float64
 5   1stFlrSF      1460 non-null   float64
 6   FullBath      1460 non-null   float64
 7   YearBuilt     1460 non-null   float64
 8   YearRemodAdd  1460 non-null   float64
 9   GarageYrBlt   1460 non-null   float64
 10  TotRmsAbvGrd  1460 non-null   float64
 11  SalePrice     1460 non-null   float64
dtypes: float64(12)
memory usage: 137.0 KB
None


In [15]:
df_selected.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,YearBuilt,YearRemodAdd,GarageYrBlt,TotRmsAbvGrd,SalePrice
0,0.651479,0.370333,0.311725,0.351000,-0.459303,-0.793434,0.789741,1.050994,0.878668,1.021157,0.912210,12.247699
1,-0.071836,-0.482512,0.311725,-0.060731,0.466465,0.257140,0.789741,0.156734,-0.429577,-0.104483,-0.318683,12.109016
2,0.651479,0.515013,0.311725,0.631726,-0.313369,-0.627826,0.789741,0.984752,0.830215,0.937776,-0.318683,12.317171
3,0.651479,0.383659,1.650307,0.790804,-0.687324,-0.521734,-1.026041,-1.863632,-0.720298,0.812705,0.296763,11.849405
4,1.374795,1.299326,1.650307,1.698485,0.199680,-0.045611,0.789741,0.951632,0.733308,0.896086,1.527656,12.429220


In [16]:
import pickle

with open('df_selected_02.pkl', 'wb') as f:
    pickle.dump(df_selected, f)

print("df_selected가 'df_selected_02.pkl' 파일로 저장되었습니다.")


df_selected가 'df_selected_02.pkl' 파일로 저장되었습니다.


In [17]:
# 1단계: 데이터 구조 확인 및 필요한 컬럼들 존재 여부 확인
print("데이터셋의 기본 정보:")
print("Shape:", df.shape)
print("\n필요한 컬럼들 존재 확인:")

required_cols = ['YrSold', 'YearBuilt', 'YearRemodAdd', 'OverallQual', 'GrLivArea', 
                'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'TotRmsAbvGrd', 'FullBath', 
                'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'GarageArea']

for col in required_cols:
    if col in df.columns:
        print(f"✓ {col}: 존재")
    else:
        print(f"✗ {col}: 없음")

print(f"\n전체 컬럼 수: {len(df.columns)}")
print("샘플 데이터:")
print(df[['YrSold', 'YearBuilt', 'YearRemodAdd', 'OverallQual', 'GrLivArea']].head())


데이터셋의 기본 정보:
Shape: (1460, 81)

필요한 컬럼들 존재 확인:
✓ YrSold: 존재
✓ YearBuilt: 존재
✓ YearRemodAdd: 존재
✓ OverallQual: 존재
✓ GrLivArea: 존재
✓ TotalBsmtSF: 존재
✓ 1stFlrSF: 존재
✓ 2ndFlrSF: 존재
✓ TotRmsAbvGrd: 존재
✓ FullBath: 존재
✓ HalfBath: 존재
✓ BsmtFullBath: 존재
✓ BsmtHalfBath: 존재
✓ GarageArea: 존재

전체 컬럼 수: 81
샘플 데이터:
   YrSold  YearBuilt  YearRemodAdd  OverallQual  GrLivArea
0    2008       2003          2003            7       1710
1    2007       1976          1976            6       1262
2    2008       2001          2002            7       1786
3    2006       1915          1970            7       1717
4    2008       2000          2000            8       2198


In [18]:
# 2단계: 7개의 새로운 엔지니어링된 피처 생성
import numpy as np

print("새로운 피처들 생성 중...")

# 1. EffectiveAge: YrSold - max(YearBuilt, YearRemodAdd)
df['EffectiveAge'] = df['YrSold'] - df[['YearBuilt', 'YearRemodAdd']].max(axis=1)

# 2. Quality_x_Area: OverallQual * GrLivArea  
df['Quality_x_Area'] = df['OverallQual'] * df['GrLivArea']

# 3. TotalSF: TotalBsmtSF + 1stFlrSF + 2ndFlrSF
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

# 4. AvgRoomSize: GrLivArea / TotRmsAbvGrd (0으로 나누기 처리)
df['AvgRoomSize'] = df['GrLivArea'] / df['TotRmsAbvGrd']
df['AvgRoomSize'] = df['AvgRoomSize'].replace([np.inf, -np.inf, np.nan], 0)

# 5. TotalBathrooms: FullBath + (0.5 * HalfBath) + BsmtFullBath + (0.5 * BsmtHalfBath)
df['TotalBathrooms'] = (df['FullBath'] + 
                       (0.5 * df['HalfBath']) + 
                       df['BsmtFullBath'] + 
                       (0.5 * df['BsmtHalfBath']))

# 6. GarageRatio: GarageArea / GrLivArea (0으로 나누기 처리)
df['GarageRatio'] = df['GarageArea'] / df['GrLivArea']
df['GarageRatio'] = df['GarageRatio'].replace([np.inf, -np.inf, np.nan], 0)

# 7. IsNewBuild: YrSold == YearBuilt (categorical로 변환)
df['IsNewBuild'] = (df['YrSold'] == df['YearBuilt']).astype('category')

print("✓ 모든 새로운 피처들이 생성되었습니다.")
print("✓ IsNewBuild 컬럼이 categorical 타입으로 설정되었습니다.")

# 생성된 피처들 확인
new_features = ['EffectiveAge', 'Quality_x_Area', 'TotalSF', 'AvgRoomSize', 
                'TotalBathrooms', 'GarageRatio', 'IsNewBuild']

print(f"\n새로운 피처들의 기본 통계:")
# categorical 변수와 수치형 변수를 분리해서 출력
numeric_features = [f for f in new_features if f != 'IsNewBuild']
print("수치형 피처들:")
print(df[numeric_features].describe())

print(f"\ncategorical 피처 (IsNewBuild):")
print(f"데이터 타입: {df['IsNewBuild'].dtype}")
print(f"유니크 값: {df['IsNewBuild'].unique()}")
print(f"값 분포:\n{df['IsNewBuild'].value_counts()}")


새로운 피처들 생성 중...
✓ 모든 새로운 피처들이 생성되었습니다.
✓ IsNewBuild 컬럼이 categorical 타입으로 설정되었습니다.

새로운 피처들의 기본 통계:
수치형 피처들:
       EffectiveAge  Quality_x_Area       TotalSF  AvgRoomSize  \
count   1460.000000     1460.000000   1460.000000  1460.000000   
mean      22.950000     9673.956164   2567.048630   230.905362   
std       20.640653     5186.744876    821.714421    44.740397   
min       -1.000000      334.000000    334.000000   120.000000   
25%        4.000000     5790.000000   2009.500000   199.000000   
50%       14.000000     8820.000000   2474.000000   226.500000   
75%       41.000000    12180.000000   3004.000000   256.107143   
max       60.000000    56420.000000  11752.000000   470.166667   

       TotalBathrooms  GarageRatio  
count     1460.000000  1460.000000  
mean         2.210616     0.323993  
std          0.785399     0.148638  
min          1.000000     0.000000  
25%          2.000000     0.246751  
50%          2.000000     0.311559  
75%          2.500000     0.402601  
m

In [19]:
# 3단계: 결측값 및 데이터 타입 확인 (categorical 변수 고려)
print("새로 생성된 피처들의 결측값 확인:")
for feature in new_features:
    missing_count = df[feature].isnull().sum()
    dtype = df[feature].dtype
    print(f"{feature}: {missing_count}개 결측값, 타입: {dtype}")

print("\n새로 생성된 피처들의 샘플 데이터:")
print(df[new_features].head())

# 무한대값 확인 (categorical 변수는 제외)
print("\n무한대값 확인:")
numeric_new_features = [f for f in new_features if df[f].dtype != 'category']
for feature in numeric_new_features:
    inf_count = np.isinf(df[feature]).sum()
    if inf_count > 0:
        print(f"{feature}: {inf_count}개 무한대값 발견")
    else:
        print(f"{feature}: 무한대값 없음")

# categorical 변수는 별도로 확인
categorical_features = [f for f in new_features if df[f].dtype == 'category']
if categorical_features:
    print(f"\ncategorical 변수들:")
    for feature in categorical_features:
        print(f"{feature}: categorical 타입 (무한대값 확인 불가)")
        print(f"  - 유니크 값: {df[feature].cat.categories.tolist()}")
        print(f"  - 결측값: {df[feature].isnull().sum()}개")


새로 생성된 피처들의 결측값 확인:
EffectiveAge: 0개 결측값, 타입: int64
Quality_x_Area: 0개 결측값, 타입: int64
TotalSF: 0개 결측값, 타입: int64
AvgRoomSize: 0개 결측값, 타입: float64
TotalBathrooms: 0개 결측값, 타입: float64
GarageRatio: 0개 결측값, 타입: float64
IsNewBuild: 0개 결측값, 타입: category

새로 생성된 피처들의 샘플 데이터:
   EffectiveAge  Quality_x_Area  TotalSF  AvgRoomSize  TotalBathrooms  \
0             5           11970     2566   213.750000             3.5   
1            31            7572     2524   210.333333             2.5   
2             6           12502     2706   297.666667             3.5   
3            36           12019     2473   245.285714             2.0   
4             8           17584     3343   244.222222             3.5   

   GarageRatio IsNewBuild  
0     0.320468      False  
1     0.364501      False  
2     0.340426      False  
3     0.373908      False  
4     0.380346      False  

무한대값 확인:
EffectiveAge: 무한대값 없음
Quality_x_Area: 무한대값 없음
TotalSF: 무한대값 없음
AvgRoomSize: 무한대값 없음
TotalBathrooms: 무한대값 없음
Garage

In [20]:
# 4단계: 최종 피처 리스트 정의 및 길이 확인
improved_feature_list_01 = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 
    'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd',
    'EffectiveAge', 'TotalBathrooms', 'Quality_x_Area', 
    'AvgRoomSize', 'IsNewBuild', 'TotalSF', 'GarageRatio'
]

print("개선된 피처 리스트:")
for i, feature in enumerate(improved_feature_list_01, 1):
    # 각 피처의 타입도 함께 표시
    if feature in df.columns:
        dtype = df[feature].dtype
        print(f"{i:2d}. {feature} ({dtype})")
    else:
        print(f"{i:2d}. {feature} (존재하지 않음)")

print(f"\n피처 리스트 길이: {len(improved_feature_list_01)}")

# 길이가 14인지 확인
if len(improved_feature_list_01) == 14:
    print("✓ 피처 리스트 길이가 올바릅니다 (14개)")
else:
    print(f"✗ 피처 리스트 길이가 예상과 다릅니다. 현재: {len(improved_feature_list_01)}개, 예상: 14개")

# 모든 피처가 df에 존재하는지 확인
print("\n피처 존재 여부 확인:")
missing_features = []
categorical_features_in_list = []
numeric_features_in_list = []

for feature in improved_feature_list_01:
    if feature in df.columns:
        dtype = df[feature].dtype
        if dtype == 'category':
            categorical_features_in_list.append(feature)
            print(f"✓ {feature} (categorical)")
        else:
            numeric_features_in_list.append(feature)
            print(f"✓ {feature} (numeric)")
    else:
        print(f"✗ {feature} - 없음")
        missing_features.append(feature)

if missing_features:
    print(f"\n경고: 다음 피처들이 데이터프레임에 없습니다: {missing_features}")
else:
    print("\n✓ 모든 피처가 데이터프레임에 존재합니다.")

print(f"\n피처 타입 요약:")
print(f"- 수치형 피처: {len(numeric_features_in_list)}개")
print(f"- categorical 피처: {len(categorical_features_in_list)}개")
print(f"- categorical 피처 목록: {categorical_features_in_list}")


개선된 피처 리스트:
 1. OverallQual (int64)
 2. GrLivArea (int64)
 3. GarageCars (int64)
 4. GarageArea (int64)
 5. TotalBsmtSF (int64)
 6. 1stFlrSF (int64)
 7. TotRmsAbvGrd (int64)
 8. EffectiveAge (int64)
 9. TotalBathrooms (float64)
10. Quality_x_Area (int64)
11. AvgRoomSize (float64)
12. IsNewBuild (category)
13. TotalSF (int64)
14. GarageRatio (float64)

피처 리스트 길이: 14
✓ 피처 리스트 길이가 올바릅니다 (14개)

피처 존재 여부 확인:
✓ OverallQual (numeric)
✓ GrLivArea (numeric)
✓ GarageCars (numeric)
✓ GarageArea (numeric)
✓ TotalBsmtSF (numeric)
✓ 1stFlrSF (numeric)
✓ TotRmsAbvGrd (numeric)
✓ EffectiveAge (numeric)
✓ TotalBathrooms (numeric)
✓ Quality_x_Area (numeric)
✓ AvgRoomSize (numeric)
✓ IsNewBuild (categorical)
✓ TotalSF (numeric)
✓ GarageRatio (numeric)

✓ 모든 피처가 데이터프레임에 존재합니다.

피처 타입 요약:
- 수치형 피처: 13개
- categorical 피처: 1개
- categorical 피처 목록: ['IsNewBuild']


In [21]:
# 5단계: 새로운 DataFrame df_improved_01 생성 (categorical 변수 고려)
print("df_improved_01 생성 중...")

# 피처 리스트에 타겟 변수 추가
columns_to_include = improved_feature_list_01 + ['SalePrice']

# 새로운 DataFrame 생성
df_improved_01 = df[columns_to_include].copy()

print("✓ df_improved_01이 성공적으로 생성되었습니다.")

# DataFrame 정보 확인
print(f"\nDataFrame 정보:")
print(f"Shape: {df_improved_01.shape}")
print(f"컬럼 수: {len(df_improved_01.columns)}")
print(f"행 수: {len(df_improved_01)}")

print(f"\n컬럼 리스트 (타입 포함):")
for i, col in enumerate(df_improved_01.columns, 1):
    dtype = df_improved_01[col].dtype
    print(f"{i:2d}. {col} ({dtype})")

# 각 컬럼의 결측값 확인
print(f"\n결측값 확인:")
missing_summary = df_improved_01.isnull().sum()
total_missing = missing_summary.sum()
print(f"전체 결측값: {total_missing}개")

if total_missing > 0:
    print("컬럼별 결측값:")
    for col in missing_summary.index:
        if missing_summary[col] > 0:
            print(f"  {col}: {missing_summary[col]}개")
else:
    print("✓ 결측값이 없습니다.")

# 데이터 타입별 피처 분류
categorical_cols = df_improved_01.select_dtypes(include=['category']).columns.tolist()
numeric_cols = df_improved_01.select_dtypes(exclude=['category']).columns.tolist()

print(f"\n데이터 타입별 분류:")
print(f"- Categorical 컬럼 ({len(categorical_cols)}개): {categorical_cols}")
print(f"- Numeric 컬럼 ({len(numeric_cols)}개): {numeric_cols}")


df_improved_01 생성 중...
✓ df_improved_01이 성공적으로 생성되었습니다.

DataFrame 정보:
Shape: (1460, 15)
컬럼 수: 15
행 수: 1460

컬럼 리스트 (타입 포함):
 1. OverallQual (int64)
 2. GrLivArea (int64)
 3. GarageCars (int64)
 4. GarageArea (int64)
 5. TotalBsmtSF (int64)
 6. 1stFlrSF (int64)
 7. TotRmsAbvGrd (int64)
 8. EffectiveAge (int64)
 9. TotalBathrooms (float64)
10. Quality_x_Area (int64)
11. AvgRoomSize (float64)
12. IsNewBuild (category)
13. TotalSF (int64)
14. GarageRatio (float64)
15. SalePrice (float64)

결측값 확인:
전체 결측값: 0개
✓ 결측값이 없습니다.

데이터 타입별 분류:
- Categorical 컬럼 (1개): ['IsNewBuild']
- Numeric 컬럼 (14개): ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'EffectiveAge', 'TotalBathrooms', 'Quality_x_Area', 'AvgRoomSize', 'TotalSF', 'GarageRatio', 'SalePrice']


In [22]:
# 6단계: df_improved_01의 첫 5행과 shape 확인 (categorical 변수 고려)
print("df_improved_01의 첫 5행:")
print(df_improved_01.head())

print(f"\ndf_improved_01의 shape: {df_improved_01.shape}")

# 수치형 변수와 categorical 변수를 분리해서 기본 통계 출력
print(f"\n수치형 변수들의 기본 통계:")
numeric_cols_only = [col for col in df_improved_01.columns if df_improved_01[col].dtype != 'category']
print(df_improved_01[numeric_cols_only].describe())

# categorical 변수들의 정보
categorical_cols_only = [col for col in df_improved_01.columns if df_improved_01[col].dtype == 'category']
if categorical_cols_only:
    print(f"\ncategorical 변수들의 정보:")
    for col in categorical_cols_only:
        print(f"\n{col}:")
        print(f"  - 데이터 타입: {df_improved_01[col].dtype}")
        print(f"  - 유니크 값: {df_improved_01[col].cat.categories.tolist()}")
        print(f"  - 값 분포:")
        value_counts = df_improved_01[col].value_counts()
        for idx, count in value_counts.items():
            percentage = (count / len(df_improved_01)) * 100
            print(f"    {idx}: {count}개 ({percentage:.1f}%)")

print(f"\n데이터 타입 요약:")
print(df_improved_01.dtypes)


df_improved_01의 첫 5행:
   OverallQual  GrLivArea  GarageCars  GarageArea  TotalBsmtSF  1stFlrSF  \
0            7       1710           2         548          856       856   
1            6       1262           2         460         1262      1262   
2            7       1786           2         608          920       920   
3            7       1717           3         642          756       961   
4            8       2198           3         836         1145      1145   

   TotRmsAbvGrd  EffectiveAge  TotalBathrooms  Quality_x_Area  AvgRoomSize  \
0             8             5             3.5           11970   213.750000   
1             6            31             2.5            7572   210.333333   
2             6             6             3.5           12502   297.666667   
3             7            36             2.0           12019   245.285714   
4             9             8             3.5           17584   244.222222   

  IsNewBuild  TotalSF  GarageRatio  SalePrice  
0   

In [26]:
# 7단계: df_improved_01을 pkl 파일로 저장 (categorical 변수 포함)
import pickle

# df_improved_01을 df_selected_03.pkl로 저장
output_file = 'df_selected_03.pkl'

with open(output_file, 'wb') as f:
        pickle.dump(df_improved_01, f)
    
print(f"✓ df_improved_01이 '{output_file}' 파일로 성공적으로 저장되었습니다.")


✓ df_improved_01이 'df_selected_03.pkl' 파일로 성공적으로 저장되었습니다.


In [28]:
# 8단계: TotalBathrooms_weighted 피처 생성 및 df_improved_02 생성

# TotalBathrooms_weighted 피처 계산
total_bathrooms_weighted = (df['FullBath'] + 
                          (0.5 * df['HalfBath']) + 
                          (0.7 * df['BsmtFullBath']) + 
                          (0.35 * df['BsmtHalfBath']))

# df_improved_01에서 TotalBathrooms 제거하고 새로운 피처 추가
df_improved_02 = df_improved_01.drop('TotalBathrooms', axis=1)
df_improved_02['TotalBathrooms_weighted'] = total_bathrooms_weighted

# df_improved_02를 df_selected_04.pkl로 저장
with open('df_selected_04.pkl', 'wb') as f:
    pickle.dump(df_improved_02, f)
    
print(f"✓ df_improved_02가 'df_selected_04.pkl'파일로 성공적으로 저장되었습니다.")

# 변경된 데이터프레임의 shape 출력
print(f"\ndf_improved_02의 shape: {df_improved_02.shape}")



✓ df_improved_02가 'df_selected_04.pkl'파일로 성공적으로 저장되었습니다.

df_improved_02의 shape: (1460, 15)


In [30]:
# 9단계: 욕실 관련 피처 재구성 및 df_improved_03 생성

# 새로운 욕실 관련 피처 계산
above_grade_bath = df['FullBath'] + (0.5 * df['HalfBath'])
basement_bath = df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])

# df_improved_02에서 TotalBathrooms_weighted 제거하고 새로운 피처 추가
df_improved_03 = df_improved_02.drop('TotalBathrooms_weighted', axis=1)
df_improved_03['AboveGradeBath'] = above_grade_bath
df_improved_03['BasementBath'] = basement_bath

# df_improved_03를 df_selected_05.pkl로 저장
with open('df_selected_05.pkl', 'wb') as f:
    pickle.dump(df_improved_03, f)
    
print(f"✓ df_improved_03가 'df_selected_05.pkl' 파일로 성공적으로 저장되었습니다.")

# 변경된 데이터프레임의 shape 출력
print(f"\ndf_improved_03의 shape: {df_improved_03.shape}")


✓ df_improved_03가 'df_selected_05.pkl' 파일로 성공적으로 저장되었습니다.

df_improved_03의 shape: (1460, 16)
